In [87]:
import pandas as pd
import math
from scipy import spatial
import libpysal as ps
import numpy as np
from pointpats import PointPattern
from pointpats import PoissonPointProcess, PoissonClusterPointProcess, Window, poly_from_bbox, PointPattern
import libpysal as ps
from libpysal.cg import shapely_ext
%matplotlib inline
import matplotlib.pyplot as plt
import pointpats.quadrat_statistics as qs
from pointpats import PointPattern, as_window
from pointpats import PoissonPointProcess as csr
from pointpats.geometry import *
from pointpats.geometry import (prepare_hull as _prepare_hull,     area as _area,
    k_neighbors as _k_neighbors,
    build_best_tree as _build_best_tree,
    prepare_hull as _prepare_hull,
    TREE_TYPES,
)
from pointpats.ripley import _prepare # very important to prepare data :)

In [65]:
def kinhom(coords_array):
    coordinates, support, distances, metric, hull, edge_correction = _prepare(
            coords_array, support=None,
        distances=None,
        metric="euclidean",
        hull=None,
        edge_correction=None,
        )

    hull = _prepare_hull(coords_array, hull = None)

    n = coordinates.shape[0]
    upper_tri_distances = spatial.distance.pdist(coordinates, metric='euclidean') # OK  DISTANCES 
    n_pairs_less_than_d = (upper_tri_distances < support.reshape(-1, 1)).sum(axis=1) # N PAIRS LESS THAN DISTANCES 
    lambda_1 = n / _area(hull) #INTENSITY 
    k_estimate = ((n_pairs_less_than_d * 2) / n) / lambda_1 # OK  ((N PAIRS LESS THAN DISTANCE * 2) / N POINTS )/ INTENSITY    #BRING UP THE POINT FROM BADDELEY THAT ITS SHOULD BE N-1\HULL AREAS (PP 204)
    uno_fratto_a = 1/ _area(hull)
    lambda_2 = (n * (n - 1)) / (_area(hull))**2
    k_inhom_estimate = ((n_pairs_less_than_d *2) / (lambda_2)) * uno_fratto_a
    return support, k_inhom_estimate

# Calculate Kinhom on simulated points 

In [79]:
simulated_companies = pd.read_pickle('./simulated_companies.pkl')
simulated_companies.columns = [['lat', 'long']]d
def coords_to_array(df): 
    df['array'] = df.apply(lambda x: np.array(list(zip(x['lat'], x['long']))), axis = 1) 
    return df
array_df = coords_to_array(simulated_companies)
list_of_arrays = array_df['array'].values.tolist()
kinhom_values = []
for x in list_of_arrays:
    for array in x: 
        kinhom_lista = kinhom(array)
        kinhom_values.append(kinhom_lista)
kinhom_estimates = pd.DataFrame(kinhom_values, columns = ['support', 'kinhom_estimate'])

# Calculate L transformation

In [96]:
def L_function(list_of_estimates):
    L = [math.sqrt(x/math.pi) for x in list_of_estimates]
    return L
kinhom_estimates['l_estimates'] = kinhom_estimates['kinhom_estimate'].apply(L_function)

In [99]:
kinhom_estimates

,support,kinhom_estimate,l_estimates
0,"[0.0, 0.005843940808605459, 0.0116878816172109...","[0.0, 0.0001255911684666141, 0.000753547010799...","[0.0, 0.006322729674776331, 0.0154874614847554..."
1,"[0.0, 0.004690301043119766, 0.0093806020862395...","[0.0, 0.0001289552533311945, 0.000386865759993...","[0.0, 0.006406850397087044, 0.0110969904042475..."
2,"[0.0, 0.004019622042128335, 0.0080392440842566...","[0.0, 0.0001336463525582218, 0.000133646352558...","[0.0, 0.006522342774777047, 0.0065223427747770..."
3,"[0.0, 0.008317952651485832, 0.0166359053029716...","[0.0, 0.00039152049783559454, 0.00091354782828...","[0.0, 0.011163549843336984, 0.0170526040605013..."
4,"[0.0, 0.012343536506338812, 0.0246870730126776...","[0.0, 0.00045996181682514176, 0.00383301514020...","[0.0, 0.012100016262902313, 0.0349297382329275..."
...,...,...,...
995,"[0.0, 0.00748284661663554, 0.01496569323327108...","[0.0, 0.00029271944182663353, 0.00146359720913...","[0.0, 0.009652745319939735, 0.0215841947048782..."
996,"[0.0, 0.010390779877309068, 0.0207815597546181...","[0.0, 0.00036711996074315203, 0.00357941961724...","[0.0, 0.010810083853511518, 0.0337544760138491..."
997,"[0.0, 0.004667899100341413, 0.0093357982006828...","[0.0, 0.00013795580038456656, 0.00048284530134...","[0.0, 0.006626665460003631, 0.0123973558840511..."
998,"[0.0, 0.0077147498361330785, 0.015429499672266...","[0.0, 0.0002095156374600483, 0.001466609462220...","[0.0, 0.00816644957822139, 0.02160639467832211..."


# Find Null L(d)s